In [0]:
from pyspark.sql.functions import col, to_timestamp, year, month, hour, lit

In [0]:
df1 = spark.table("bronze.spotify.streaming_history_audio_2024_2025_10")

df1_filtred = df1.filter(df1["ts"] >= "2025-01-01") 

In [0]:
df2 = spark.table("bronze.spotify.streaming_history_audio_2025_11")

df_union = df1_filtred.unionByName(df2, allowMissingColumns=True)

df_union = df_union.withColumn('ts', to_timestamp(col('ts')))
df_union = df_union.withColumn('year', year(col('ts')))
df_union = df_union.withColumn('month', month(col('ts')))
df_union = df_union.withColumn('hour', hour(col('ts')))
df_union = df_union.withColumn('minutes_played', col('ms_played') / 60000)
df_union = df_union.withColumn('hours_played', col('ms_played') / 3600000)
df_union = df_union.withColumn('username', lit("namiiara"))

columns_to_check = [
    'master_metadata_track_name',
    'master_metadata_album_artist_name',
    'master_metadata_album_album_name',
    'spotify_track_uri'
]
df_nona = df_union.dropna(subset=columns_to_check)
df_stringnull = df_nona.filter(
    (col("master_metadata_track_name") != 'null') &
    (col("master_metadata_album_artist_name") != 'null') &
    (col("master_metadata_album_album_name") != 'null') &
    (col("spotify_track_uri") != 'null')
)

df_cast = df_stringnull.withColumn("offline", col("offline").cast("string")) \
                      .withColumn("skipped", col("skipped").cast("string")) \
                      .withColumn("offline_timestamp", col("offline_timestamp").cast("string"))

df_2025 = df_cast.select(
    "ts",
    "username",
    "platform",
    "ms_played",
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "spotify_track_uri",
    "reason_start",
    "reason_end",
    "shuffle",
    "skipped",
    "offline",
    "offline_timestamp",
    "incognito_mode",
    "year",
    "month",
    "hour",
    "minutes_played",
    "hours_played"
)

df_2025.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("silver.spotify_eng.streaming_history_202509")

In [0]:
display(df_2025)

In [0]:
display(df_2025.schema)

print("\n")
display(df_2014_2024.schema)

In [0]:
df_2014_2024 = spark.table("silver.spotify_eng.spotify_analise_2014_2024")

df_2014_2025 = df_2025.unionByName(df_2014_2024, allowMissingColumns=False)

df_2014_2025.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("silver.spotify_eng.streaming_history_2014_202509")